# Localise sources in counts map

This notebook uses deep learning to explore source localisation with CTA.

## Gather data and label info

First we gather all info of simulations and relative counts maps.

In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile, expandvars

source = 'crab'
simdata = f'{expandvars("$HOME")}/E4/z60_NLST/{source}/dl3_random_irf/merged_simulator_data.dat'
fitspath = f'{expandvars("$HOME")}/E4/z60_NLST/{source}/dl3_random_irf'

fitsfiles = [join(fitspath, f) for f in listdir(fitspath) if '.fits' in f and isfile(join(fitspath, f))]
fitsfiles = sorted(fitsfiles)
f"Dataset size: {len(fitsfiles)}"

### Simulations info

In [ ]:
siminfo = pd.read_csv(simdata, sep=' ', header=0).sort_values('seed')
siminfo.rename(columns={'computation_time': 'sim_time'}, inplace=True)
siminfo.reset_index(inplace=True, drop=True)
siminfo.head()

In [ ]:
siminfo.iloc[1]

In [ ]:
datainfo = siminfo

## Gather dataset

Now we gather all data from the fits files and convert to numpy arrays. Here we need to set some parameters such as the exposure time we want to use (for now it will be fixed exposure time).

In [ ]:
def extract_heatmap(data, trange, smoothing, nbins):
    data = data[(data['TIME'] >= trange[0]) & (data['TIME'] <= trange[1])] 
    
    ra = data['RA'].to_numpy()
    dec = data['DEC'].to_numpy()
    
    heatmap, xe, ye = np.histogram2d(ra, dec, bins=nbins)
    heatmap = gaussian_filter(heatmap, sigma=smoothing)
    return heatmap.T

def set_wcs(point_ra, point_dec, point_ref, pixelsize):
    w = WCS(naxis=2)
    w.wcs.ctype = ['RA---CAR', 'DEC--CAR']
    w.wcs.cunit = ['deg', 'deg']
    w.wcs.crpix = [point_ref, point_ref]
    w.wcs.crval = [point_ra, point_dec]
    w.wcs.cdelt = [pixelsize, pixelsize]
    #w.wcs.lonpole = 0.0
    #w.wcs.latpole = 67.49
    return w

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from os.path import basename
from astropy.io import fits
from scipy.ndimage import gaussian_filter
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization import astropy_mpl_style
from astropy import units as u
from astropy.wcs import WCS
from matplotlib.colors import SymLogNorm
from astropy.table import Table, vstack
plt.style.use(astropy_mpl_style)

trange = [0, 50]
smoothing = 3
pixelsize = 0.02
nbins = int(5/pixelsize)
sample = 10

for f in fitsfiles[:sample]:
    name = basename(f).replace('.fits','')
    print(f"Simulation: {name}")

    point_ra = datainfo[datainfo['name'] == name]['point_ra'].to_numpy()[0]
    point_dec = datainfo[datainfo['name'] == name]['point_dec'].to_numpy()[0]
    print(f"Point: [{point_ra}, {point_dec}]")
    
    heatmap = Table.read(f, hdu=1).to_pandas()
    heatmap = extract_heatmap(heatmap, trange, smoothing, nbins)
    wcs = set_wcs(point_ra=point_ra, point_dec=point_dec, point_ref=nbins/2+0.5, pixelsize=pixelsize)

    plt.figure()
    ax = plt.subplot(projection=wcs)
    ax.coords[0].set_format_unit(u.deg)
    ax.coords[1].set_format_unit(u.deg)
    ax.invert_yaxis()
    ax.invert_xaxis()

    plt.imshow(heatmap)
    #plt.imshow(heatmap, norm=SymLogNorm(1, base=10))
    plt.colorbar()
    
    plt.scatter(point_ra, point_dec, c='k', marker='+', s=50, transform=ax.get_transform('world')) 
    if 'background' not in name:
        source_ra = datainfo[datainfo['name'] == name]['source_ra'].to_numpy()[0]
        source_dec = datainfo[datainfo['name'] == name]['source_dec'].to_numpy()[0]
        print(f"Source: [{source_ra}, {source_dec}]")
        plt.scatter(source_ra * u.deg, source_dec * u.deg, c='none', marker='o', s=200, transform=ax.get_transform('world'), edgecolor='w', facecolor='none')
    
